In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.linear_model import LinearRegression

In [5]:
noise517 = pd.read_csv('https://raw.githubusercontent.com/WilliamJCole/IS362_Final_Project/master/Noise_Complaints_April_2017.csv',sep=',')
noise517

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Latitude.1,Longitude.1,Location.1
0,35835573,4/1/17 0:01,4/1/17 0:28,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10467.0,3395 RESERVOIR OVAL WEST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.879145,-73.876464,"(40.879144669133176, -73.87646438522717)"
1,35838142,4/1/17 0:04,4/1/17 0:12,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,10455.0,627 WALES AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.814966,-73.906500,"(40.81496598157052, -73.90650015950133)"
2,35833590,4/1/17 0:05,4/1/17 9:37,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,11101.0,1-50 50 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.743642,-73.958878,"(40.74364166035955, -73.95887755839249)"
3,35834330,4/1/17 0:05,4/1/17 8:49,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,11226.0,2114 ALBEMARLE ROAD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.647413,-73.958865,"(40.64741283883138, -73.9588647959135)"
4,35838814,4/1/17 0:05,4/1/17 4:55,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Store/Commercial,10003.0,321 1 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.734140,-73.980400,"(40.73413951274722, -73.98039971548371)"
5,35838072,4/1/17 0:06,4/1/17 1:26,NYPD,New York City Police Department,Noise - Commercial,Loud Music/Party,Store/Commercial,10036.0,251 WEST 48 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.760583,-73.985922,"(40.76058322950115, -73.98592204392392)"
6,35844431,4/1/17 0:07,4/4/17 22:03,DEP,Department of Environmental Protection,Noise,Noise: air condition/ventilation equipment (NV1),NaN,10452.0,957 WOODYCREST AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.831527,-73.928863,"(40.8315271048226, -73.92886303221528)"
7,35838767,4/1/17 0:07,4/1/17 9:33,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,11374.0,98-30 67 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.725042,-73.855020,"(40.72504161746349, -73.85502020259588)"
8,35833644,4/1/17 0:07,4/1/17 3:47,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,11224.0,2850 STILLWELL AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.578074,-73.981759,"(40.5780742789993, -73.98175946661651)"
9,35835078,4/1/17 0:07,4/1/17 1:38,NYPD,New York City Police Department,Noise - Commercial,Banging/Pounding,Store/Commercial,10128.0,1361 LEXINGTON AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.782225,-73.953582,"(40.78222524276318, -73.95358180659666)"
